In [3]:
%pip install -U langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_upstage import UpstageDocumentParseLoader
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

loader = UpstageDocumentParseLoader(
        "./캠퍼스 마켓 기획서.pdf", 
        output_format="markdown"
    )
docs = loader.load()

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256
)

splits = text_splitter.split_documents(docs)

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large-passage"
)

vectorstore = FAISS.from_documents(splits, embeddings)

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [15]:
vectorstore.save_local("faiss_index")
embeddings = UpstageEmbeddings(model="embedding-query")
# 로컬에서 로드
new_vectorstore = FAISS.load_local(
    "faiss_index", 
    embeddings, 
    allow_dangerous_deserialization=True # 필수 설정 (신뢰할 수 있는 파일인 경우)
)

In [29]:
retriever = new_vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.0, # 0.0 ~ 1.0 (유사도 기준, 높을수록 엄격)
        "k": 3
    }
)

In [30]:
# 2. 검색 실행 (동기)
query = "해당 프로젝트의 기획의도"
docs = retriever.invoke(query)

# 3. 결과 확인
for doc in docs:
    print(f"[내용]: {doc.page_content}")
    print(f"[메타데이터]: {doc.metadata}")

[내용]: # 캠퍼스 마켓 (Uni-Market) 프로젝트 기획안팀 데브루키 (DevRookies)# 2024년 2월 15일차 례1 1. 프로젝트 개요 (Overview) 2
1.1 1.1 프로젝트 소개 · · 2
1.2 1.2 팀원 구성 (Team DevRookies) 2
1.3 1.3 개발 기간 · · · 2
2 2. 시장 분석 및 타겟 유저 3
2.1 2.1 페르소나 (Persona) 3
2.2 2.2 경쟁사 분석 · · 3
3 3. 핵심 기능 (Key Features) 3
3.1 3.1 사용자 기능 · 3
3.2 3.2 상품 거래 기능 4
3.3 3.3 커뮤니케이션 4
4 4. 유저 플로우 (User Flow) 5
5 5. 데이터베이스 설계 (ERD Draft) 5
6 6. 화면 설계 (Wireframe Concept) 6
6.1 6.1 메인 페이지 (Main) · · · 6
6.2 6.2 상품 상세 페이지 (Detail) 6
7 7. 기술 스택 (Tech Stack) 6
8 8. 개발 일정 (WBS) 7
9 9. 예상되는 어려움 및 해결 방안 (Troubleshooting Plan) 81# 1 1. 프로젝트 개요 (Overview)# 1.1 1.1 프로젝트 소개한 줄 요약: 우리 학교 학생들끼리 믿고 거래하는 중고 전공 서적 & 자취 물품 거래 플랫폼대학생들의 경우 매 학기 비싼 전공 서적을 구매해야 하는 부담이 있지만, 기존 중고 거래 플랫
폼(당근마켓, 중고나라 등)은 '학교 인증' 절차가 없어 직거래 시 신뢰도 문제가 발생하거나, 캠퍼스
내에서의 거래만을 필터링하기 어려운 불편함이 있었습니다. Uni-Market은 학교 이메일 인증을
통해 "진짜 우리 학교 학생"들만 거래할 수 있는 폐쇄형 커뮤니티 마켓입니다.# 기획 의도"왜 전공책은 한 학기만 쓰고 버려질까?"
[메타데이터]: {'total_pages': 8, 'coordinates': [[{'x': 0.2059, 'y': 0.1291}, {'x': 0.7956, 'y': 0.1291}, {'